In [6]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

In [4]:
import os
## Either get environment variables, or a fallback name, which is the second parameter.
## Currently, fill in the fallback values. Later on, we will make sure to work with Environment values. So we're already preparing for it in here!
workspace_name = os.environ.get('WORKSPACE', 'profit-model-ws')
subscription_id = os.environ.get('SUBSCRIPTION_ID', '3edc5525-afb7-4ed7-a865-5aa28c23bcc6')
resource_group = os.environ.get('RESOURCE_GROUP', 'smurf-profit-ml-gr')

In [7]:
# The credential "DefaultAzureCredential" will use the same name as your logged in user.
credential = DefaultAzureCredential()

In [8]:
ml_client = MLClient(
    credential, subscription_id, resource_group, workspace_name
)

In [22]:
import pandas as pd

df = pd.read_csv("azureml://subscriptions/3edc5525-afb7-4ed7-a865-5aa28c23bcc6/resourcegroups/smurf-profit-ml-gr/workspaces/profit-model-ws/datastores/workspaceblobstore/paths/UI/2025-06-09_085107_UTC/train_V2_cleaned.csv")

DF_PATH= "azureml://subscriptions/3edc5525-afb7-4ed7-a865-5aa28c23bcc6/resourcegroups/smurf-profit-ml-gr/workspaces/profit-model-ws/datastores/workspaceblobstore/paths/UI/2025-06-09_085107_UTC/train_V2_cleaned.csv"

df

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


,income_am,profit_last_am,profit_am,damage_am,damage_inc,crd_lim_rec,credit_use_ic,gluten_ic,lactose_ic,insurance_ic,...,score_pos,score_neg,tenure_started_in_season,dining_ic_False,dining_ic_True,dining_ic_missing,presidential_False,presidential_True,presidential_missing,is_return_customer
0,-0.910345,-1.153916,0.331249,1.275968,6.350261,2.572171,False,False,False,False,...,0.716841,0.152289,False,True,False,False,True,False,False,False
1,-0.780435,-0.148628,-0.626860,-0.251395,-0.396167,-0.560258,False,False,False,True,...,0.390596,1.418531,False,True,False,False,True,False,False,True
2,-0.737825,-0.043537,-0.634849,-0.251395,-0.396167,-0.560258,False,False,False,True,...,-1.547549,-1.763662,True,True,False,False,True,False,False,True
3,-0.910345,-1.153916,-0.624207,-0.251395,-0.396167,-0.725122,False,False,False,False,...,0.482201,1.780404,True,True,False,False,True,False,False,False
4,1.352095,1.058226,0.431958,1.098807,1.852643,2.352351,False,False,True,False,...,-0.413334,0.706683,False,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4942,-0.170999,0.192906,-0.551771,-0.251395,-0.396167,-0.725122,False,False,False,False,...,0.609486,-1.219401,False,True,False,False,True,False,False,True
4943,-0.910345,-1.153916,-0.682781,-0.251395,-0.396167,-0.725122,False,False,False,False,...,0.215214,-0.325199,True,True,False,False,True,False,False,False
4944,-0.870040,-0.483348,-0.319246,-0.251395,-0.396167,-0.175573,False,False,False,True,...,-0.083084,-1.409122,False,True,False,False,True,False,False,True
4945,0.301303,0.393887,1.364007,0.591407,1.852643,2.572171,False,False,False,True,...,0.377917,-0.504463,False,True,False,False,True,False,False,True


# Set environment

In [ ]:
from azure.ai.ml.entities import Environment
import os

pipeline_job_env = Environment(
    name="aml-Smurf-Train",
    description="Env for training Smurf model",
    conda_file=os.path.join("conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
ml_client.environments.create_or_update(pipeline_job_env)


# training component

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output
import os

training_component = command(
    name="smurf_train_model",
    display_name="Train Smurf Model",
    inputs={
        "data": Input(type="uri_file"),  # your cleaned_dataset.csv
    },
    outputs={
        "model_output": Output(type="uri_folder", mode="rw_mount"),
    },
    code=os.path.join("components", "training", "code"),
    command="""python train.py \
        --data ${{inputs.data}} \
        --model_output ${{outputs.model_output}}""",
    environment="aml-Smurf-Train@latest",
)
ml_client.create_or_update(training_component.component)
